In [26]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import csv
import re


In [27]:
urls = [
        'https://www.flipkart.com/search?q=refrigerator&otracker=AS_Query_HistoryAutoSuggest_4_0&otracker1=AS_Query_HistoryAutoSuggest_4_0&marketplace=FLIPKART&as-show=on&as=off&as-pos=4&as-type=HISTORY'
        ]

In [28]:
driver = webdriver.Chrome()

In [29]:
# List to store scraped data before writing to CSV
data_list = []

for url in urls:
    driver.get(url)
    time.sleep(4)
    print(f"Scraping the data from {url}")

    last_page = None  

    while True:
        # Get page source and parse
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # Extract current page number
        page_info = soup.find("div", class_="_1G0WLw")
        if page_info:
            page_text = page_info.find("span").text  
            current_page = page_text.split()[1]  
            print(f"📄 Scraping details from Page {current_page}...")

            if current_page == last_page:  
                print("✅ Scraping completed. No more pages available.")
                break

            last_page = current_page  

        # Extract the data from the HTML
        title = soup.find_all('div', class_="DOjaWF gdgoEp")
        for i in title:
            products = i.find_all('div', class_="cPHDOP col-12-12")

            # Exclude the last two products (pagination elements)
            products = products[:-2]  

            for product in products:
                j = product.find_all('div', class_="tUxRFH")
                for k in j:
                    link = k.find('a', class_="CGtC98")
                    product_url = "https://www.flipkart.com" + link['href'] if link else "N/A"

                    name = k.find('div', class_="KzDlHZ").text.strip()
                    rating_tag = k.find('div', class_="XQDdHH")
                    rating = rating_tag.get_text(strip=True) if rating_tag else "N/A"

                    ratings_reviews = k.find('span', class_="Wphh3N")
                    num_rating, num_reviews = "N/A", "N/A"

                    if ratings_reviews:
                        text = ratings_reviews.get_text(strip=True)  
                        parts = text.split("&")  

                        if len(parts) == 2:  
                            num_rating = ''.join(filter(str.isdigit, parts[0]))  
                            num_reviews = ''.join(filter(str.isdigit, parts[1]))  

                    
                    # Extract refrigerator specifications
                    details = k.find('div', class_="_6NESgJ")
                    compressor, stabilizer, warranty = "N/A", "N/A", "N/A"

                    if details:
                        specifications = details.find_all('li', class_="J+igdf")
                        for spec in specifications:
                            spec_text = spec.text.strip()

                            # Extract compressor type (but avoid warranty mentions)
                            if re.search(r"\bcompressor\b", spec_text, re.IGNORECASE) and not re.search(r"warranty", spec_text, re.IGNORECASE):
                                compressor = spec_text  # Keep original format
        
                            # Extract stabilizer presence
                            if re.search(r"built-in stabilizer", spec_text, re.IGNORECASE):
                                stabilizer = "Yes"

                            # Extract warranty details
                            warranty_match = re.search(r"(\d+\s*Year.*?)$", spec_text, re.IGNORECASE)
                            if warranty_match:
                                warranty = warranty_match.group(1)

                    # Output extracted details
                    print(f"Compressor: {compressor}, Built-in Stabilizer: {stabilizer}, Warranty: {warranty}")
    
                    price = k.find('div', class_="Nx9bqj _4b5DiR")
                    price = price.text.strip().replace("₹", "").replace(",", "") if price else "N/A"

                    original_price = k.find('div', class_="yRaY8j ZYYwLA")
                    original_price = original_price.text.strip().replace("₹", "").replace(",", "") if original_price else "N/A"

                    discount = k.find('div', class_="UkUFwK")
                    discount = discount.text.strip().replace("off", "").replace("%", "") if discount else "N/A"

                    data_list.append([
                    product_url, name, rating, num_rating, num_reviews,
                    compressor, stabilizer, warranty,  # ✅ Correct variable names
                    price, original_price, discount
                    ])


                    print(f"✅ Scraped: {name}")

        # Locate the "Next" button
        try:
            next_button = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.XPATH, "//a[@class='_9QVEpD' and span[text()='Next']]"))
            )

            # Scroll to the "Next" button
            driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", next_button)
            time.sleep(2)

            # Click the "Next" button
            next_button.click()
            print("➡️ Moving to the next page...\n")
            time.sleep(2)  
        except Exception as e:
            print("✅ Scraping completed or no more pages available.", e)
            break  


Scraping the data from https://www.flipkart.com/search?q=refrigerator&otracker=AS_Query_HistoryAutoSuggest_4_0&otracker1=AS_Query_HistoryAutoSuggest_4_0&marketplace=FLIPKART&as-show=on&as=off&as-pos=4&as-type=HISTORY
📄 Scraping details from Page 1...
Compressor: Reciprocating Compressor, Built-in Stabilizer: Yes, Warranty: 1 Year Comprehensive Warranty on Product and  9 Years Warranty on Compressor
✅ Scraped: Whirlpool 215 L Frost Free Triple Door Refrigerator
Compressor: Fixed speed compressor, Built-in Stabilizer: Yes, Warranty: 1 Year Comprehensive Warranty on Product and 10 Years Warranty on Compressor
✅ Scraped: Godrej 180 L Direct Cool Single Door 4 Star Refrigerator  with Large Vegetable Tray And 2.25 L Bottle ...
Compressor: Digital Inverter Compressor, Built-in Stabilizer: Yes, Warranty: 1 year comprehensive warranty on the product, 20 years on compressor by the brand.
✅ Scraped: SAMSUNG 183 L Direct Cool Single Door 4 Star Refrigerator with Base Drawer  with Digital Inverter


In [30]:
# ✅ Write the data to CSV at the end
csv_filename = "flipkart_refrigerators.csv"
with open(csv_filename, "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    
    # Write header with relevant refrigerator specifications
    writer.writerow([
        "Product URL", "Name", "Rating", "Number of Ratings", "Number of Reviews", 
        "Compressor Type", "Built-in Stabilizer", "Warranty", 
        "Price (₹)", "Original Price (₹)", "Discount (%)"
    ])

    # Write all data rows
    writer.writerows(data_list)

print(f"📂 Data successfully saved in '{csv_filename}'")


📂 Data successfully saved in 'flipkart_refrigerators.csv'
